In [1]:
import datetime
import random
import time
import requests
import pandas as pd

In [2]:
def get_date_range(start_date, end_date):
    if start_date > end_date:
        raise ValueError("Start date cannot be after end date")
    num_days = (end_date - start_date).days + 1
    date_list = [start_date + datetime.timedelta(days=x) for x in range(num_days)]
    return date_list

In [3]:
def format_date_for_api(date_str_ddmmyyyy):
    try:
        dt_obj = datetime.datetime.strptime(date_str_ddmmyyyy, '%d/%m/%Y')
        return dt_obj.strftime('%m/%d/%Y')
    except ValueError:
        return None

In [4]:
def format_bytes(byte_count):
    if byte_count < 1024:
        return f"{byte_count} Bytes"
    elif byte_count < 1024**2:
        return f"{byte_count / 1024:.2f} KB"
    elif byte_count < 1024**3:
        return f"{byte_count / (1024**2):.2f} MB"
    else:
        return f"{byte_count / (1024**3):.2f} GB"

In [5]:
def scrape_cafef_foreign_trading(exchange, date_str_ddmmyyyy, session):
    api_date = format_date_for_api(date_str_ddmmyyyy)
    if not api_date:
        print(f"  -> Skipping: Invalid date format '{date_str_ddmmyyyy}' for {exchange}.")
        return None, None, 0

    target_exchange = exchange.upper()
    referer_exchange = exchange.lower()
    api_url = "https://cafef.vn/du-lieu/Ajax/PageNew/DataGDNN/GDNuocNgoai.ashx"
    params = {'TradeCenter': target_exchange, 'Date': api_date}
    headers = {'Referer': f'https://cafef.vn/du-lieu/tracuulichsu2/3/{referer_exchange}/{date_str_ddmmyyyy}.chn'}

    response_bytes = 0

    try:
        response = session.get(api_url, params=params, headers=headers, timeout=30)
        response_bytes = len(response.content)
        response.raise_for_status()
        data = response.json()

        if not data or 'Data' not in data or not data['Data'] or 'ListDataNN' not in data['Data']:
            summary_check = data.get('Data', {})
            if summary_check and (summary_check.get('BuyVolume') is not None or summary_check.get('SellVolume') is not None):
                print(f"  -> OK (Summary Only): No detailed records found for {target_exchange} on {date_str_ddmmyyyy}.")
                summary = {
                    'Date': date_str_ddmmyyyy, 'Exchange': target_exchange,
                    **{key: summary_check.get(key) for key in [
                        'Symbol', 'BuyVolume', 'BuyVolumePercent', 'BuyValue', 'BuyValuePercent',
                        'SellVolume', 'SellVolumePercent', 'SellValue', 'SellValuePercent',
                        'DiffVolume', 'DiffValue'
                    ]}
                }
                return summary, pd.DataFrame(), response_bytes
            else:
                print(f"  -> No data found for {target_exchange} on {date_str_ddmmyyyy}.")
                return None, None, response_bytes

        summary = {
            'Date': date_str_ddmmyyyy, 'Exchange': target_exchange,
            **{key: data['Data'].get(key) for key in [
                'Symbol', 'BuyVolume', 'BuyVolumePercent', 'BuyValue', 'BuyValuePercent',
                'SellVolume', 'SellVolumePercent', 'SellValue', 'SellValuePercent',
                'DiffVolume', 'DiffValue'
            ]}
        }
        details_list = data['Data'].get('ListDataNN', [])
        details_df = pd.DataFrame(details_list) if details_list else pd.DataFrame()
        if not details_df.empty:
            details_df['Date'] = date_str_ddmmyyyy
            details_df['Exchange'] = target_exchange

        return summary, details_df, response_bytes

    except requests.exceptions.RequestException as e:
        print(f"  -> Error: Request failed for {target_exchange} on {date_str_ddmmyyyy}: {e}")
        return None, None, response_bytes
    except ValueError as e:
        print(f"  -> Error: JSON Parsing failed for {target_exchange} on {date_str_ddmmyyyy}: {e}")
        return None, None, response_bytes

In [6]:
def scrape_cafef_foreign_trading_multi(exchanges, dates_ddmmyyyy, min_delay_seconds, max_delay_seconds):
    all_summaries = []
    all_details_dfs = []
    total_bytes_downloaded = 0

    num_requests = len(dates_ddmmyyyy) * len(exchanges)
    print(f"Starting scrape for {num_requests} total requests.")
    print(f"Random delay: {min_delay_seconds:.1f}s - {max_delay_seconds:.1f}s")

    with requests.Session() as session:
        session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)',
            'Accept': 'application/json, text/javascript, */*; q=0.01',
            'X-Requested-With': 'XMLHttpRequest'
        })

        request_count = 0
        for date_str in dates_ddmmyyyy:
            for exchange in exchanges:
                request_count += 1
                print(f"\n[Request {request_count}/{num_requests}: {exchange} @ {date_str}]")
                summary_data, details_df, bytes_received = scrape_cafef_foreign_trading(exchange, date_str, session)
                total_bytes_downloaded += bytes_received

                if summary_data:
                    all_summaries.append(summary_data)
                if isinstance(details_df, pd.DataFrame) and not details_df.empty:
                    all_details_dfs.append(details_df)

                if request_count < num_requests:
                    delay = random.uniform(min_delay_seconds, max_delay_seconds)
                    print(f"Sleeping {delay:.2f}s...")
                    time.sleep(delay)

    summary_df = pd.DataFrame(all_summaries) if all_summaries else pd.DataFrame()
    details_df = pd.concat(all_details_dfs, ignore_index=True) if all_details_dfs else pd.DataFrame()

    return summary_df, details_df, total_bytes_downloaded

In [7]:
def run_cafef_scrape(
    exchanges, dates_ddmmyyyy,
    min_delay_seconds=0.6, max_delay_seconds=3.0,
    save_details_csv=True, output_filename=None,
    print_summary=False, print_details_head=10, print_details_tail=5
):
    print("="*20 + " Starting CafeF Scrape Run " + "="*20)

    if not exchanges or not dates_ddmmyyyy:
        print("Error: Exchanges and dates list cannot be empty.")
        return pd.DataFrame(), pd.DataFrame()

    # Convert datetime.date to 'DD/MM/YYYY'
    if isinstance(dates_ddmmyyyy[0], datetime.date):
        dates_ddmmyyyy = [d.strftime("%d/%m/%Y") for d in dates_ddmmyyyy]

    summary_df, details_df, total_bytes = scrape_cafef_foreign_trading_multi(
        exchanges, dates_ddmmyyyy, min_delay_seconds, max_delay_seconds
    )

    print("\n" + "="*20 + " Scrape Complete " + "="*20)
    print(f"Estimated Total Downloaded: {format_bytes(total_bytes)}")

    if print_summary and not summary_df.empty:
        print("\n--- Summary Data ---")
        print(summary_df)

    if not details_df.empty:
        if print_details_head > 0:
            print("\n--- Details (Head) ---")
            print(details_df.head(print_details_head))
        if print_details_tail > 0:
            print("\n--- Details (Tail) ---")
            print(details_df.tail(print_details_tail))

        if save_details_csv:
            if output_filename is None:
                start_str = dates_ddmmyyyy[0].replace("/", "-")
                end_str = dates_ddmmyyyy[-1].replace("/", "-")
                output_filename = f'cafef_foreign_details_{start_str}_to_{end_str}.csv'
            try:
                details_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
                print(f"\nSaved to {output_filename}")
            except Exception as e:
                print(f"Error saving file: {e}")

    else:
        print("\nNo detailed data retrieved.")

    print("="*20 + " Run Finished " + "="*20)
    return summary_df, details_df

In [8]:
my_exchanges = ['HOSE', 'HNX', 'UPCOM', 'HASTC']
start = datetime.date(2024, 3, 12)
end = datetime.date(2025, 3, 12)
my_dates = get_date_range(start, end)

summary_results, details_results = run_cafef_scrape(
    exchanges=my_exchanges,
    dates_ddmmyyyy=my_dates,
    save_details_csv=True,
    print_details_head=5,
    print_details_tail=5
)

==================== Starting CafeF Scrape Run ====================
Starting scrape for 1464 total requests.
Random delay: 0.6s - 3.0s

[Request 1/1464: HOSE @ 12/03/2024]
Sleeping 1.17s...

[Request 2/1464: HNX @ 12/03/2024]
Sleeping 1.15s...

[Request 3/1464: UPCOM @ 12/03/2024]
Sleeping 1.66s...

[Request 4/1464: HASTC @ 12/03/2024]
Sleeping 0.96s...

[Request 5/1464: HOSE @ 13/03/2024]
Sleeping 2.07s...

[Request 6/1464: HNX @ 13/03/2024]
Sleeping 0.93s...

[Request 7/1464: UPCOM @ 13/03/2024]
Sleeping 2.70s...

[Request 8/1464: HASTC @ 13/03/2024]
Sleeping 1.78s...

[Request 9/1464: HOSE @ 14/03/2024]
Sleeping 2.01s...

[Request 10/1464: HNX @ 14/03/2024]
Sleeping 1.95s...

[Request 11/1464: UPCOM @ 14/03/2024]
Sleeping 1.64s...

[Request 12/1464: HASTC @ 14/03/2024]
Sleeping 2.87s...

[Request 13/1464: HOSE @ 15/03/2024]
Sleeping 2.32s...

[Request 14/1464: HNX @ 15/03/2024]
Sleeping 2.11s...

[Request 15/1464: UPCOM @ 15/03/2024]
Sleeping 0.91s...

[Request 16/1464: HASTC @ 15/0

In [9]:
pd.set_option('display.float_format', '{:,.2f}'.format)
def basic_stats(df, name):
    print(name)
    display(df.head())
    display(df.tail())
    print(df.info())
    print("Số giá trị bị rỗng:")
    print(df.isnull().sum())
    print("Thống kê các giá trị:")
    display(df.describe())
    print(df.nunique())
    print("Các giá trị độc nhất của từng trường dữ liệu:")
    for col in list(df.columns):
        print(f"{col}:")
        print(f"{list(df[col].unique())[:20]}..., số giá trị độc nhất: {len(list(df[col].unique()))}")
    print("Các bản ghi bị trùng:")
    display(df[df.duplicated()])

In [10]:
basic_stats(details_results,"Thị trường đầu tư nước ngoài")

Thị trường đầu tư nước ngoài


,Symbol,TradeDate,BuyValue,SellValue,BuyVolume,SellVolume,Room,TotalRoom,Percent,NetVolume,NetValue,BasicPrice,ClosePrice,AveragePrice,Date,Exchange
0,EIB,/Date(1710176400000)/,70993520000,250470000,3915700,13800,468965024,0,3.13,3901900,70743050000,0.00,0.00,0.00,12/03/2024,HOSE
1,VRE,/Date(1710176400000)/,90619740000,32896680000,3635300,1322200,450208352,0,29.67,2313100,57723060000,0.00,0.00,0.00,12/03/2024,HOSE
2,STB,/Date(1710176400000)/,92779725000,41159545000,3058000,1357800,111048472,0,24.11,1700200,51620180000,0.00,0.00,0.00,12/03/2024,HOSE
3,HAH,/Date(1710176400000)/,66075085000,1556355000,1519100,36400,27294636,0,4.13,1482700,64518730000,0.00,0.00,0.00,12/03/2024,HOSE
4,HCM,/Date(1710176400000)/,38598215000,11288530000,1383800,406000,33455340,0,42.65,977800,27309685000,0.00,0.00,0.00,12/03/2024,HOSE


,Symbol,TradeDate,BuyValue,SellValue,BuyVolume,SellVolume,Room,TotalRoom,Percent,NetVolume,NetValue,BasicPrice,ClosePrice,AveragePrice,Date,Exchange
489495,SSI,/Date(1741712400000)/,31556130000,85249360000,1181600,3192200,1210044418,0,38.38,-2010600,-53693230000,0.00,0.00,0.00,12/03/2025,HASTC
489496,VPB,/Date(1741712400000)/,9538440000,50186230000,487000,2557600,406965109,0,24.87,-2070600,-40647790000,0.00,0.00,0.00,12/03/2025,HASTC
489497,FPT,/Date(1741712400000)/,74590280000,369669650000,545200,2706300,71217375,0,44.16,-2161100,-295079370000,0.00,0.00,0.00,12/03/2025,HASTC
489498,TPB,/Date(1741712400000)/,7489440000,44616795000,462800,2760900,48417323,0,28.17,-2298100,-37127355000,0.00,0.00,0.00,12/03/2025,HASTC
489499,VIX,/Date(1741712400000)/,12807655000,50143030000,1043400,4105600,1376573985,0,5.62,-3062200,-37335375000,0.00,0.00,0.00,12/03/2025,HASTC


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 489500 entries, 0 to 489499
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Symbol        489500 non-null  object 
 1   TradeDate     489500 non-null  object 
 2   BuyValue      489500 non-null  int64  
 3   SellValue     489500 non-null  int64  
 4   BuyVolume     489500 non-null  int64  
 5   SellVolume    489500 non-null  int64  
 6   Room          489500 non-null  int64  
 7   TotalRoom     489500 non-null  int64  
 8   Percent       489500 non-null  float64
 9   NetVolume     489500 non-null  int64  
 10  NetValue      489500 non-null  int64  
 11  BasicPrice    489500 non-null  float64
 12  ClosePrice    489500 non-null  float64
 13  AveragePrice  489500 non-null  float64
 14  Date          489500 non-null  object 
 15  Exchange      489500 non-null  object 
dtypes: float64(4), int64(8), object(4)
memory usage: 59.8+ MB
None
Số giá trị bị rỗng:
Symbol   

,BuyValue,SellValue,BuyVolume,SellVolume,Room,TotalRoom,Percent,NetVolume,NetValue,BasicPrice,ClosePrice,AveragePrice
count,"489,500.00","489,500.00","489,500.00","489,500.00","489,500.00","489,500.00","489,500.00","489,500.00","489,500.00","489,500.00","489,500.00","489,500.00"
mean,"1,400,012,209.26","1,746,426,034.17","42,543.31","53,559.32","48,978,092.03",126.66,4.41,"-11,016.01","-346,413,824.91",0.41,0.41,0.20
std,"10,465,722,104.76","13,194,594,273.88","304,043.95","420,506.83","154,568,263.47","42,036.85",23.38,"374,877.58","11,003,175,567.02",4.84,4.84,3.32
min,0.00,0.00,0.00,0.00,"-1,501,712,808.00",0.00,0.00,"-109,942,542.00","-1,627,241,121,600.00",0.00,0.00,0.00
25%,0.00,0.00,0.00,0.00,"44,100.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,0.00,0.00,0.00,0.00,"5,733,259.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00
75%,0.00,"1,475,000.00",0.00,100.00,"27,939,093.75",0.00,0.79,0.00,0.00,0.00,0.00,0.00
max,"704,573,040,000.00","1,627,241,121,600.00","40,326,700.00","109,942,542.00","1,767,241,646.00","20,000,000.00","1,822.18","14,654,600.00","589,443,800,000.00",385.10,384.00,210.00


Symbol           1644
TradeDate         447
BuyValue        52128
SellValue       56444
BuyVolume        9704
SellVolume      11114
Room            87644
TotalRoom           4
Percent         12255
NetVolume       15305
NetValue        69226
BasicPrice       1309
ClosePrice       1343
AveragePrice     3870
Date              249
Exchange            4
dtype: int64
Các giá trị độc nhất của từng trường dữ liệu:
Symbol:
['EIB', 'VRE', 'STB', 'HAH', 'HCM', 'OCB', 'VIC', 'GVR', 'NLG', 'SZC', 'AAA', 'CTG', 'KDH', 'VHM', 'FTS', 'IDI', 'FCN', 'HDB', 'VSC', 'VHC']..., số giá trị độc nhất: 1644
TradeDate:
['/Date(1710176400000)/', '/Date(1710262800000)/', '/Date(1710349200000)/', '/Date(1710435600000)/', '/Date(1710694800000)/', '/Date(1710781200000)/', '/Date(1710867600000)/', '/Date(1710954000000)/', '/Date(1711040400000)/', '/Date(1711299600000)/', '/Date(1711386000000)/', '/Date(1711472400000)/', '/Date(1711558800000)/', '/Date(1711645200000)/', '/Date(1711904400000)/', '/Date(1711990800000)/'

,Symbol,TradeDate,BuyValue,SellValue,BuyVolume,SellVolume,Room,TotalRoom,Percent,NetVolume,NetValue,BasicPrice,ClosePrice,AveragePrice,Date,Exchange


In [11]:
# Loại bỏ TradeDate, chỉnh định dạng Date, đặt làm index
details_results = details_results.drop(columns = 'TradeDate')
details_results['Date'] = pd.to_datetime(details_results['Date'], dayfirst=True)
details_results.head()

,Symbol,BuyValue,SellValue,BuyVolume,SellVolume,Room,TotalRoom,Percent,NetVolume,NetValue,BasicPrice,ClosePrice,AveragePrice,Date,Exchange
0,EIB,70993520000,250470000,3915700,13800,468965024,0,3.13,3901900,70743050000,0.00,0.00,0.00,2024-03-12,HOSE
1,VRE,90619740000,32896680000,3635300,1322200,450208352,0,29.67,2313100,57723060000,0.00,0.00,0.00,2024-03-12,HOSE
2,STB,92779725000,41159545000,3058000,1357800,111048472,0,24.11,1700200,51620180000,0.00,0.00,0.00,2024-03-12,HOSE
3,HAH,66075085000,1556355000,1519100,36400,27294636,0,4.13,1482700,64518730000,0.00,0.00,0.00,2024-03-12,HOSE
4,HCM,38598215000,11288530000,1383800,406000,33455340,0,42.65,977800,27309685000,0.00,0.00,0.00,2024-03-12,HOSE


In [12]:
# Loại bỏ các dòng trùng lặp (khác sàn những giá trị như nhau)
details_results = details_results[~details_results.drop(columns='Exchange').duplicated()]

In [13]:
details_results = details_results.set_index('Date')
basic_stats(details_results,"Thị trường đầu tư nước ngoài")

Thị trường đầu tư nước ngoài


,Symbol,BuyValue,SellValue,BuyVolume,SellVolume,Room,TotalRoom,Percent,NetVolume,NetValue,BasicPrice,ClosePrice,AveragePrice,Exchange
Date,,,,,,,,,,,,,,
2024-03-12,EIB,70993520000,250470000,3915700,13800,468965024,0,3.13,3901900,70743050000,0.00,0.00,0.00,HOSE
2024-03-12,VRE,90619740000,32896680000,3635300,1322200,450208352,0,29.67,2313100,57723060000,0.00,0.00,0.00,HOSE
2024-03-12,STB,92779725000,41159545000,3058000,1357800,111048472,0,24.11,1700200,51620180000,0.00,0.00,0.00,HOSE
2024-03-12,HAH,66075085000,1556355000,1519100,36400,27294636,0,4.13,1482700,64518730000,0.00,0.00,0.00,HOSE
2024-03-12,HCM,38598215000,11288530000,1383800,406000,33455340,0,42.65,977800,27309685000,0.00,0.00,0.00,HOSE


,Symbol,BuyValue,SellValue,BuyVolume,SellVolume,Room,TotalRoom,Percent,NetVolume,NetValue,BasicPrice,ClosePrice,AveragePrice,Exchange
Date,,,,,,,,,,,,,,
2025-03-12,VEA,396430000,1613640000,9900,40300,627155970,0,1.80,-30400,-1217210000,0.00,0.00,0.00,UPCOM
2025-03-12,SGP,0,1822560000,0,50000,104800485,0,0.55,-50000,-1822560000,0.00,0.00,0.00,UPCOM
2025-03-12,ACV,7854300000,26829580000,76200,261300,992294368,0,3.42,-185100,-18975280000,0.00,0.00,0.00,UPCOM
2025-03-12,QNS,49940000,9976050000,1000,200000,133896008,0,12.58,-199000,-9926110000,0.00,0.00,0.00,UPCOM
2025-03-12,DDV,0,13695070000,0,700000,68912951,0,1.83,-700000,-13695070000,0.00,0.00,0.00,UPCOM


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 387070 entries, 2024-03-12 to 2025-03-12
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Symbol        387070 non-null  object 
 1   BuyValue      387070 non-null  int64  
 2   SellValue     387070 non-null  int64  
 3   BuyVolume     387070 non-null  int64  
 4   SellVolume    387070 non-null  int64  
 5   Room          387070 non-null  int64  
 6   TotalRoom     387070 non-null  int64  
 7   Percent       387070 non-null  float64
 8   NetVolume     387070 non-null  int64  
 9   NetValue      387070 non-null  int64  
 10  BasicPrice    387070 non-null  float64
 11  ClosePrice    387070 non-null  float64
 12  AveragePrice  387070 non-null  float64
 13  Exchange      387070 non-null  object 
dtypes: float64(4), int64(8), object(2)
memory usage: 44.3+ MB
None
Số giá trị bị rỗng:
Symbol          0
BuyValue        0
SellValue       0
BuyVolume       0
SellVolum

,BuyValue,SellValue,BuyVolume,SellVolume,Room,TotalRoom,Percent,NetVolume,NetValue,BasicPrice,ClosePrice,AveragePrice
count,"387,070.00","387,070.00","387,070.00","387,070.00","387,070.00","387,070.00","387,070.00","387,070.00","387,070.00","387,070.00","387,070.00","387,070.00"
mean,"911,806,297.51","1,136,494,639.81","27,779.03","35,118.88","37,119,447.66",80.09,3.36,"-7,339.86","-224,688,342.30",0.37,0.37,0.13
std,"8,396,900,109.50","10,773,311,607.22","243,958.11","371,772.41","129,647,119.34","33,427.02",19.19,"339,194.95","9,066,369,249.89",4.70,4.70,2.64
min,0.00,0.00,0.00,0.00,"-1,501,712,808.00",0.00,0.00,"-109,942,542.00","-1,627,241,121,600.00",0.00,0.00,0.00
25%,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
50%,0.00,0.00,0.00,0.00,"4,378,465.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00
75%,0.00,0.00,0.00,0.00,"19,190,601.00",0.00,0.39,0.00,0.00,0.00,0.00,0.00
max,"704,573,040,000.00","1,627,241,121,600.00","40,326,700.00","109,942,542.00","1,767,241,646.00","20,000,000.00","1,822.18","14,654,600.00","589,443,800,000.00",385.10,384.00,210.00


Symbol           1644
BuyValue        52128
SellValue       56444
BuyVolume        9704
SellVolume      11114
Room            87644
TotalRoom           4
Percent         12255
NetVolume       15305
NetValue        69226
BasicPrice       1309
ClosePrice       1343
AveragePrice     3870
Exchange            3
dtype: int64
Các giá trị độc nhất của từng trường dữ liệu:
Symbol:
['EIB', 'VRE', 'STB', 'HAH', 'HCM', 'OCB', 'VIC', 'GVR', 'NLG', 'SZC', 'AAA', 'CTG', 'KDH', 'VHM', 'FTS', 'IDI', 'FCN', 'HDB', 'VSC', 'VHC']..., số giá trị độc nhất: 1644
BuyValue:
[70993520000, 90619740000, 92779725000, 66075085000, 38598215000, 13162050000, 43979810000, 23426855000, 50972520000, 30186035000, 7083250000, 34827015000, 30453170000, 46955880000, 32800890000, 6378830000, 8777650000, 38633655000, 9344760000, 31133310000]..., số giá trị độc nhất: 52128
SellValue:
[250470000, 32896680000, 41159545000, 1556355000, 11288530000, 935290000, 9839165000, 1229855000, 21704700000, 435750000, 179180000, 13886830000,

,Symbol,BuyValue,SellValue,BuyVolume,SellVolume,Room,TotalRoom,Percent,NetVolume,NetValue,BasicPrice,ClosePrice,AveragePrice,Exchange
Date,,,,,,,,,,,,,,
2024-03-13,PNC,0,0,0,0,5351533,0,0.53,0,0,0.00,0.00,0.00,HOSE
2024-03-13,NAF,0,0,0,0,49955224,0,20.61,0,0,0.00,0.00,0.00,HOSE
2024-03-13,PMG,0,0,0,0,13353937,0,20.18,0,0,0.00,0.00,0.00,HOSE
2024-03-13,PSH,0,0,0,0,-100,0,0.00,0,0,0.00,0.00,0.00,HOSE
2024-03-13,PTL,0,0,0,0,-78584,0,0.08,0,0,0.00,0.00,0.00,HOSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-12,HDP,0,0,0,0,-14506,0,0.15,0,0,0.00,0.00,0.00,UPCOM
2025-03-12,HEC,0,0,0,0,-100,0,0.00,0,0,0.00,0.00,0.00,UPCOM
2025-03-12,HDS,0,0,0,0,2529832,0,0.00,0,0,0.00,0.00,0.00,UPCOM
